# Cuaderno 25: Uso de heurísticas
## Heurística de inserción más cercana y heurística 3-OPT para el ATSP

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo dirigido **completo** $D=(V,A)$; y,
* un vector $c \in \ZZ^{A}$ de costos asociados a los arcos de $D$.

Consideraremos la formulación del *problema del agente viajero asimétrico (Asymmetric Traveling Salesman Problem, ATSP)* empleando desigualdades de corte: 

\begin{align*}
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{(i,j) \in \delta^{+}(i) } x_{ij} = 1, \quad \forall i \in V,\\
&\sum_{(j,i) \in \delta^{-}(i) } x_{ji} = 1, \quad \forall i \in V,\\
&\sum_{ij \in \delta^{+}(W)} x_{ij} \geq 1, \quad \forall W \subset V, \emptyset \neq W \neq V,\\
& x_{ij} \in \{0, 1\}, \quad \forall (i,j) \in A,
\end{align*}
donde $\delta^{+}(W) := \{ (i,j) \in A \, : \, i \in W, j \not\in W\}$.

En el Cuaderno 29 se implementó este modelo utilizando una función callback para separar dinámicamente las desigualdades de corte, tanto para soluciones enteras como para soluciones fraccionarias.

Una posibilidad para acelerar la solución de modelos de programación entera computacionalmente difíciles consiste en inicializar el solver con una solución factible de buena calidad, la cual puede encontrarse por métodos heurísticos. 

Una *heurística constructiva* es un algoritmo eficiente que produce una solución factible para un problema de optimización. La solución encontrada generalmente no es óptima, aunque se espera que su calidad sea "buena". Para el caso del ATSP, implementaremos en este cuaderno la **Heurística de la Mejor Inserción (Best-Insertion Heuristic)**, la misma que se describe a continuación:

1. Elegir $s, t \in V$ arbitrarios y construir el *tour parcial* $T=(s, t, s)$. Definir $W := V \setminus \{s, t\}$.
2. Mientras $W \neq \emptyset$, hacer:
  1. Para cada nodo $i \in W$ y para cada $k \in K:= \{1, \ldots, \card{T}-1 \}$, calcular $c_{ik}$ como el incremento en la longitud del tour $T$ al insertar el nodo $i$ entre las posiciones $k$ y $k+1$.
  2. Determinar $i^*$ y $k^*$ tales que $c_{i^*k^*} = \min_{i \in W} \min_{i \in K} \{ c_{ik}\}$.
  3. Insertar el nodo $i^*$ entre las posiciones $k^*$ y $k^* + 1$ del tour $T$.
  4. Actualizar $W:= W \setminus\{i^*\}$

Una *heurística de mejoramiento local* toma como entrada una solución factible y construye una nueva solución factible de mejor calidad realizando movimientos locales que mejoren el valor de la función objetivo. Por ejemplo, una heurística local para el ATSP es la heurística **3-OPT**:

Dado un tour factible $T$, seleccionar tres arcos $a_1$, $a_2$ y $a_3$ que no tengan nodos comunes entre sí, y eliminar estos arcos del tour. Al hacerlo, el tour $T$ se divide en tres caminos simples disjuntos $T_1$, $T_2$ y $T_3$. Suponer que el orden en el que estos caminos son visitados en $T$ es $(T_1, T_2, T_3)$, y observar que existen tres arcos disjuntos $b_1$, $b_2$ y $b_3$ que permiten recombinar los caminos en un nuevo tour $T^*$ donde estos son visitados en el orden $(T_1, T_3, T_2)$. Puede verse que la diferencia entre la longitud de los dos toures es $c(T^*) - c(T) = c(b_1) + c(b_2) + c(b_3) - c(a_1) - c(a_2) - c(a_3)$. El algoritmo consiste en realizar este tipo de sustituciones mientras sea posible mejorar el valor de la función objetivo. 

Vamos a utlizar las heurísticas de mejor inserción y 3-OPT para inyectar una solución inicial en nuestro modelo.

Definimos primero los datos. Usaremos la función `randint` del módulo random para generar valores aleatorios en el rango {0,..,100} para las coordenadas de cada nodo. Los costos de los arcos serán iguales a las distancias euclideanas entre sus nodos extremos. 

In [ ]:
from gurobipy import *
import random as rm
import math 

# iniciar generador de numeros aleatorios
rm.seed(0)

# Numero de nodos del grafo
n = 200

# Nodos del grafo
V = tuplelist(range(1,n+1))

# Posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
A = c.keys()


Emplearemos el módulo `matplotlib` para graficar el tour de la solución. Definiremos para ello la función `dibujarTour` que recibe tres argumentos: una lista `coordx` con las coordenadas horizontales de los nodos, una lista `coordy` con las coordenadas verticales y un vector `tour` con una permutación de los nodos indicando el orden de visita en la solución.

In [ ]:
import matplotlib.pyplot as plt
import random

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)
    display(plt.show())


Para la generación dinámica de las desigualdades de corte, necesitaremos calcular el corte saliente de capacidad mínima en un grafo dirigido. Emplearemos para ello la siguiente función descrita en el Cuaderno 28:

In [ ]:
import networkx as nx

# corte saliente de capacidad minima
def corte_sal_minimo(V, u):
    # Parametros:
    # V: lista con los nodos del grafo
    # u: diccionario indexado por los arcos del grafo, con sus capacidades
    # se fija un nodo s arbitrario
    s = V[0]
    V2 = [i for i in V if i!=s]
    G = nx.DiGraph()
    G.add_weighted_edges_from([(i, j, u[i,j]) for (i,j) in u.keys()], weight='u')
    # inicializar dmin con la suma de capacidades de todos los arcos
    dmin = sum([u[i,j] for (i, j) in u.keys()])
    hay_solucion = False
    # se calculan los cortes minimos (s, t) y (t, s) para todo t != s
    for t in V2: 
        d1, (W1, W1c) = nx.minimum_cut(G, s, t, capacity='u')
        d2, (W2, W2c) = nx.minimum_cut(G, t, s, capacity='u')
        if d1 < d2 and d1 < dmin:
            dmin, Wmin, Wminc = d1, W1, W1c
            hay_solucion = True
        elif d2 <= d1 and d2 < dmin:
            dmin, Wmin, Wminc = d2, W2, W2c
            hay_solucion = True
    if  not hay_solucion:
        print('*** Error:')
        print(V)
        for (i,j) in [(i, j) for (i, j) in u.keys() if i==1]:
            print([(1,j), u[1,j]])
    return dmin, Wmin, Wminc


Definimos ahora el objeto modelo, las variables y la función objetivo. 

Definimos las variables miembro `_x`, `_V` y `_A` dentro del objeto modelo para tener acceso a las variables de selección de arcos, la lista de nodos y la lista de arcos, respectivamente. 

In [ ]:
# Crear el objeto modelo
m = Model('atsp-corte-lazy-heuristics')

# Crear las variables
x = m.addVars(A, name="x", vtype=GRB.BINARY)

# Crear la funcion objetivo
m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

# Crear variables en el objeto modelo para tener acceso a x, V y A
m._x = x
m._V = V
m._A = A

Añadimos las restricciones de grado. 

In [ ]:
# Restricciones de grado saliente
m.addConstrs((x.sum(i,'*') == 1 for i in V), "g_saliente");

# Restricciones de grado entrante
m.addConstrs((x.sum('*', i) == 1 for i in V), "g_entrante");



Definimos una función callback para separar dinámicamente las restricciones de corte. Esta función actúa cada vez que el solver encuentra una nueva solución entera que satisface las restricciones de grado (`where == GRB.Callback.MIPSOL`), o cuando termina el procesamiento de un nodo del árbol de branch-and-bound y se dispone de una solución fraccionaria a la relajación lineal problema (`where == GRB.Callback.MIPNODE`). 

Cuando se dispone de una solución entera, la función accede a los valores de las variables a través del llamado a `cbGetSolution`. A partir de estos valores, la función implementa un algoritmo de marcaje para determinar el conjunto W de todos los nodos que pueden ser alcanzados desde el nodo 1. Si este conjunto es distinto de V, el mismo está asociado a una desigualdad de corte violada por `x`.

Cuando la función callback es llamada luego del procesamiento de un nodo del árbol de branch-and-bound (`where == GRB.Callback.MIPNODE`), es necesario en primer lugar verificar que se dispone de una solución fraccionaria, llamando a la función `cbGet(GRB.Callback.MIPNODE_STATUS)` y comparando su valor de retorno con `GRB.OPTIMAL`. En este caso, los valores de la solución fraccionaria son recuperados llamando a la función `cbGetNodeRel`. Estos valores se usan como capacidades en los arcos del grafo del problema. Se emplean las funciones descritas en el Cuaderno 30 para calcular un corte de capacidad mínimima en el grafo. Si la capacidad de este corte es inferior a 1, se separa la desigualdad correspondiente.

In [ ]:
# Funcion callback para separar desigualdades de corte
def mycallback(model, where):
    # Esta funcion se activara cuando se encuentre una nueva solucion entera
    if where == GRB.Callback.MIPSOL:
        # Recuperar los valores de la solucion actual
        vx = model.cbGetSolution(model._x)
        # Determinar los arcos seleccionados en la solucion
        L = tuplelist([(i,j) for (i,j) in model._A if vx[i,j]>0.1])
        # Construir la lista W de nodos que pueden ser alcanzados desde 1
        W = [1]
        i = 1
        while True :
            # seleccionar el único arco saliente de i en L
            a = L.select(i,'*')[0]
            L.remove(a)
            if a[1]==1: 
                break
            W.append(a[1])
            i = a[1]
        # Si W!=V, agregar la desigualdad de corte asociada a W
        if len(W)!=len(model._V):
            Wc = [i for i in model._V if i not in W]
            model.cbLazy(model._x.sum(W, Wc) >= 1)
    # Esta funcion se activara cuando se encuentre la solucion optima en un nodo
    elif where == GRB.Callback.MIPNODE:
        if model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.OPTIMAL:
            # Recuperar los valores de la solucion (fraccionaria) actual
            vx = model.cbGetNodeRel(model._x)
            # Crear diccionario de capacidades (=valores de x) indexado por los arcos
            D={}
            for (i,j) in model._A:
                D[i,j] = vx[i,j]
            # encontrar el corte saliente de capacidad minima
            u, W, Wc = corte_sal_minimo(model._V, D)
            # Si la capacidad de este corte es inferior a 1, agregar nueva desigualdad lazy
            if u <= 0.99:
                model.cbLazy(model._x.sum(W, Wc) >= 1)

Para poder utilizar restricciones tipo *lazy* en un modelo, debe fijarse el parámetro `LazyConstraints` al valor de 1.

In [ ]:
# Configurar Gurobi para usar restricciones lazy
m.Params.LazyConstraints = 1


Fijamos un límite para el tiempo de cálculo y una tolerancia para la brecha de optimalidad:

In [ ]:
# Terminar al alcanzar un Gap del 10%
m.Params.MIPGap = 0.1

# Terminar luego de 180 segundos
m.Params.TimeLimit = 180

Construimos un tour $T$ empleando la heurística de mejor inserción (*Best-Insertion Heuristic*): 

In [ ]:
# Heurística de mejor insercion (best-insertion)
# construir una lista T con el orden de visita de los nodos en el tour
# Tour inicial:
T = [1, 2, 1]
# Nodos por procesar:
W = [i for i in V if not i in [1,2]]
# Constante suficientemente alta
# Repetir mientras W contenga nodos
while W!=[]:
    # construir una lista L con tuplas (c_i, j_i, i) donde:
    # c_i : mejor costo de insercion de i en el tour T
    # j_i : mejor posicion de insercion de i en el tour T, 1 <= j_i <= |T|-1
    L = []
    for i in W:
        # construimos primero una lista L_i con tuplas (c_ik, k)
        # c_ik: costo de insertar i en la posicion k
        Li = [(c[T[k-1],i] + c[i, T[k]] - c[T[k-1], T[k]], k) for k in range(1, len(T)-1)]
        # agregamos el elemento de Li de costo minimo a L
        # recordar que para ordenar tuplas se emplear por defecto el criterio lexicográfico
        ci, ji = min(Li)
        L.append((ci, ji, i))
    # determinar el elemento de insercion mas barata, y su posicion
    (delta, j, i) = min(L)
    # insertar i en T en la posicion j
    T.insert(j, i)
    # eliminar i de W
    W.remove(i)

print(T)
print(sum([c[T[i],T[i+1]] for i in range(0, len(T)-1)]))
       

Mejoramos localmente el tour $T$ usando una heurística 3-OPT:

In [ ]:
# Mejorar T usando 3-OPT
# este lazo se repite mientras se encuentren mejoras
hay_mejoras = True
while hay_mejoras:
    hay_mejoras = False
    print('*')
    for k1 in range(1, len(T)-5):
        for k2 in range(k1+2, len(T)-3):
            for k3 in range(k2+2, len(T)-1):
                # determinar variacion del costo al ejecutar el intercambio:
                delta = c[T[k1], T[k2+1]] + c[T[k2], T[k3+1]] + c[T[k3], T[k1+1]] \
                      - c[T[k1], T[k1+1]] - c[T[k2], T[k2+1]] - c[T[k3], T[k3+1]]
                # si la variacion es negativa, realizar el intercambio
                if delta < -0.01:
                    T = T[0:k1+1] + T[k2+1:k3+1] + T[k1+1:k2+1] + T[k3+1:len(T)]
                    hay_mejoras = True
            
print (T)
print(sum([c[T[i],T[i+1]] for i in range(0, len(T)-1)]))


Insertamos el tour $T$ como una solución inicial en el modelo. Para ello, fijamos el atributo `Start` de cada variable al valor correspondiente en la solución asociada a $T$. 

Al terminar de fijar los valores iniciales para cada variable, es necesario llamar al método `update` del objeto modelo.

In [ ]:
# originalmente, fijamos el valor de inicio de todas las variables a cero:
for i,j in A:
    x[i,j].setAttr('Start', 0.0)

# luego  fijar a 1 las variables asociadas a los arcos del tour T
for k in range(1, len(T)):
    x[T[k-1], T[k]].setAttr('Start', 1.0)

# es necesario llamar a update() luego de cambiar los atributos de variables, de restricciones o del modelo
m.update()
    

Finalmente, resolvemos el modelo y mostramos la solución, en caso de que se haya encontrado al menos una solución factible (si `m.SolCount` es mayor a cero).

In [ ]:
# Calcular la solución óptima
m.optimize(mycallback)

# Escribir la solución
if m.SolCount > 0:
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('\nTour optimo:')
    for i,j in A:
        if vx[i,j] > 0:
            print('{} --> {}'.format(i, j))


Si encontramos alguna solución factible, el siguiente fragmento de código nos permite graficarla.

In [ ]:
# Crear lista con arcos seleccionados en la solucion
vx = m.getAttr('x', x)
L = tuplelist([(i,j) for i,j in A if vx[i,j]>0.1])
# Recuperar el tour como un ordenamiento de los nodos
T = [1]
i = 1
while True:
    # Determinar sucesor de i
    a = L.select(i,'*')[0]
    L.remove(a)
    # Colocar sucesor en la lista del tour y actualizar i
    T.append(a[1])
    i = a[1]
    # Terminar cuando el nodo colocado sea 1
    if i==1: 
        break
        
# Graficar el tour
dibujarTour(coordx, coordy, T)    

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos lineales enteros
# Problema del agente viajero asimétrico (ATSP)
# Implementación con desigualdades lazy y heurísticas para una solución inicial

# Luis M. Torres (EPN 2022)

from gurobipy import *
import random as rm
import math 
import networkx as nx

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)
    display(plt.show())

# Función para calcular un corte saliente de capacidad minima
def corte_sal_minimo(V, u):
    # Parametros:
    # V: lista con los nodos del grafo
    # u: diccionario indexado por los arcos del grafo, con sus capacidades
    # se fija un nodo s arbitrario
    s = V[0]
    V2 = [i for i in V if i!=s]
    G = nx.DiGraph()
    G.add_weighted_edges_from([(i, j, u[i,j]) for (i,j) in u.keys()], weight='u')
    # inicializar dmin con la suma de capacidades de todos los arcos
    dmin = sum([u[i,j] for (i, j) in u.keys()])
    hay_solucion = False
    # se calculan los cortes minimos (s, t) y (t, s) para todo t != s
    for t in V2: 
        d1, (W1, W1c) = nx.minimum_cut(G, s, t, capacity='u')
        d2, (W2, W2c) = nx.minimum_cut(G, t, s, capacity='u')
        if d1 < d2 and d1 < dmin:
            dmin, Wmin, Wminc = d1, W1, W1c
            hay_solucion = True
        elif d2 <= d1 and d2 < dmin:
            dmin, Wmin, Wminc = d2, W2, W2c
            hay_solucion = True
    if  not hay_solucion:
        print('*** Error:')
        print(V)
        for (i,j) in [(i, j) for (i, j) in u.keys() if i==1]:
            print([(1,j), u[1,j]])
    return dmin, Wmin, Wminc

# Función callback para separar desigualdades de corte
def mycallback(model, where):
    # Esta funcion se activara cuando se encuentre una nueva solucion entera
    if where == GRB.Callback.MIPSOL:
        # Recuperar los valores de la solucion actual
        vx = model.cbGetSolution(model._x)
        # Determinar los arcos seleccionados en la solucion
        L = tuplelist([(i,j) for (i,j) in model._A if vx[i,j]>0.1])
        # Construir la lista W de nodos que pueden ser alcanzados desde 1
        W = [1]
        i = 1
        while True :
            # seleccionar el único arco saliente de i en L
            a = L.select(i,'*')[0]
            L.remove(a)
            if a[1]==1: 
                break
            W.append(a[1])
            i = a[1]
        # Si W!=V, agregar la desigualdad de corte asociada a W
        if len(W)!=len(model._V):
            Wc = [i for i in model._V if i not in W]
            model.cbLazy(model._x.sum(W, Wc) >= 1)
    # Esta funcion se activara cuando se encuentre la solucion optima en un nodo
    elif where == GRB.Callback.MIPNODE:
        if model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.OPTIMAL:
            # Recuperar los valores de la solucion (fraccionaria) actual
            vx = model.cbGetNodeRel(model._x)
            # Crear diccionario de capacidades (=valores de x) indexado por los arcos
            D={}
            for (i,j) in model._A:
                D[i,j] = vx[i,j]
            # encontrar el corte saliente de capacidad minima
            u, W, Wc = corte_sal_minimo(model._V, D)
            # Si la capacidad de este corte es inferior a 1, agregar nueva desigualdad lazy
            if u <= 0.99:
                model.cbLazy(model._x.sum(W, Wc) >= 1)
                
# iniciar generador de numeros aleatorios
rm.seed(0)

# Numero de nodos del grafo
n = 100

# Nodos del grafo
V = tuplelist(range(1,n+1))

# Posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
A = c.keys()


try:
    # Crear el objeto modelo
    m = Model('atsp-corte-lazy')

    # Crear las variables
    x = m.addVars(A, name="x", vtype=GRB.BINARY)

    # Crear la funcion objetivo
    m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

    # Crear variables en el objeto modelo para tener acceso a x, V y A
    m._x = x
    m._V = V
    m._A = A

    # Restricciones de grado saliente
    m.addConstrs((x.sum(i,'*') == 1 for i in V), "g_saliente");

    # Restricciones de grado entrante
    m.addConstrs((x.sum('*', i) == 1 for i in V), "g_entrante");

    # Escribir el modelo a un archivo
    # m.write('atsp-corte-lazy.lp')

    # Configurar Gurobi para usar restricciones lazy
    m.Params.LazyConstraints = 1

    # Terminar al alcanzar un Gap del 10%
    m.Params.MIPGap = 0.1

    # Terminar luego de 180 segundos
    m.Params.TimeLimit = 180

    # Heuristica de mejor insercion (best-insertion)
    # construir una lista T con el orden de visita de los nodos en el tour
    # Tour inicial:
    T = [1, 2, 1]
    # Nodos por procesar:
    W = [i for i in V if not i in [1,2]]
    # Constante suficientemente alta
    # Repetir mientras W contenga nodos
    while W!=[]:
        # construir una lista L con tuplas (c_i, j_i, i) donde:
        # c_i : mejor costo de insercion de i en el tour T
        # j_i : mejor posicion de insercion de i en el tour T, 1 <= j_i <= |T|-1
        L = []
        for i in W:
            # construimos primero una lista L_i con tuplas (c_ik, k)
            # c_ik: costo de insertar i en la posicion k
            Li = [(c[T[k-1],i] + c[i, T[k]] - c[T[k-1], T[k]], k) for k in range(1, len(T)-1)]
            # agregamos el elemento de Li de costo minimo a L
            ci, ji = min(Li)
            L.append((ci, ji, i))
        # determinar el elemento de insercion mas barata, y su posicion
        (delta, j, i) = min(L)
        # insertar i en T en la posicion j
        T.insert(j, i)
        # eliminar i de W
        W.remove(i)

    print(T)
    print(sum([c[T[i],T[i+1]] for i in range(0, len(T)-1)]))

    # Heuristica de mejoramiento local 3-OPT
    # este lazo se repite mientras se encuentren mejoras
    hay_mejoras = True
    while hay_mejoras:
        hay_mejoras = False
        for k1 in range(1, len(T)-5):
            for k2 in range(k1+2, len(T)-3):
                for k3 in range(k2+2, len(T)-1):
                    # determinar variacion del costo al ejecutar el intercambio:
                    delta = c[T[k1], T[k2+1]] + c[T[k2], T[k3+1]] + c[T[k3], T[k1+1]] \
                          - c[T[k1], T[k1+1]] - c[T[k2], T[k2+1]] - c[T[k3], T[k3+1]]
                    # si la variacion es negativa, realizar el intercambio
                    if delta < -0.01:
                        T = T[0:k1+1] + T[k2+1:k3+1] + T[k1+1:k2+1] + T[k3+1:len(T)]
                        hay_mejoras = True
            
    print (T)
    print(sum([c[T[i],T[i+1]] for i in range(0, len(T)-1)]))
    
    # inyectar solucion inicial en el solver
    # originalmente, fijamos el valor de inicio de todas las variables a cero:
    for i,j in A:
        x[i,j].setAttr('Start', 0.0)

    # luego  fijar a 1 las variables asociadas a los arcos del tour T
    for k in range(1, len(T)):
        x[T[k-1], T[k]].setAttr('Start', 1.0)

    # es necesario llamar a update() luego de cambiar los atributos de variables, de restricciones o del modelo
    m.update()

    # Calcular la solucion optima
    m.optimize(mycallback)

    # Escribir la solucion
    if m.getAttr(GRB.Attr.SolCount) > 0:
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('\nTour óptimo:')
        for i,j in A:
            if vx[i,j] > 0:
                print('{} --> {}'.format(i, j))

        # Recuperar el tour como un ordenamiento de los nodos
        L = tuplelist([(i,j) for i,j in A if vx[i,j]>0.1])
        T = [1]
        i = 1
        while True:
            # Determinar sucesor de i
            a = L.select(i,'*')[0]
            L.remove(a)
            # Colocar sucesor en la lista del tour y actualizar i
            T.append(a[1])
            i = a[1]
            # Terminar cuando el nodo colocado sea 1
            if i==1: 
                break
        # Graficar el tour
        dibujarTour(coordx, coordy, T)        
        
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')